In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [40]:
url = "https://godeacs.com/sports/mens-basketball/roster/2015-16"
page = requests.get(url)
players = BeautifulSoup(page.content, 'html.parser').find_all(class_ = "s-person-card s-person-card--list flex flex-col overflow-hidden rounded-[10px] border s-person-card--theme-light-theme shadow-level-1 border")

all_players = []
for player in players: 
    player_info = {}
    player_info["name"] = player.find(class_ = "hover:underline focus:underline").text
    attrs = player.find(class_ = "s-person-details__bio-stats s-text-details s-text-details-bold py-0.5").find_all(class_ = "s-person-details__bio-stats-item")
    player_info["Position"] = (attrs[0].contents[-1])
    player_info["Academic Year"] = (attrs[1].contents[-1])
    try:
        player_info["Height"] = (attrs[2].contents[-1])
    except:
        player_info["Height"] = "NA"
    try:
        player_info["Weight"] = (attrs[3].contents[-1])
    except:
        player_info["Weight"] = "NA"
    
    locations =  player.find_all(class_ = "s-person-card__content__person__location-item")
    player_info["Hometown"] = locations[0].contents[-1].text
    try:
        player_info["High School"] = locations[1].contents[-1].text.split('/')[0]
    except:
        player_info["High School"] = "NA"
    all_players.append(player_info)

In [86]:
def get_roster(url):
    page = requests.get(url)
    players = BeautifulSoup(page.content, 'html.parser').find_all(class_ = "s-person-card s-person-card--list flex flex-col overflow-hidden rounded-[10px] border s-person-card--theme-light-theme shadow-level-1 border")

    all_players = []
    for player in players: 
        player_info = {}
        player_info["name"] = player.find(class_ = "hover:underline focus:underline").text
        attrs = player.find(class_ = "s-person-details__bio-stats s-text-details s-text-details-bold py-0.5").find_all(class_ = "s-person-details__bio-stats-item")
        try:
            player_info["Position"] = (attrs[0].contents[-1])
        except:
            player_info["Position"] = "NA"
        try:
            player_info["Academic Year"] = (attrs[1].contents[-1])
        except:
            player_info["Academic Year"] = "NA"
        try:
            player_info["Height"] = (attrs[2].contents[-1])
        except:
            player_info["Height"] = "NA"
        try:
            player_info["Weight"] = (attrs[3].contents[-1])
        except:
            player_info["Weight"] = "NA"

        locations =  player.find_all(class_ = "s-person-card__content__person__location-item")
        try:
            player_info["Hometown"] = locations[0].contents[-1].text
        except:
            player_info["Hometown"] = 'NA'
        try:
            player_info["High School"] = locations[1].contents[-1].text.split('/')[0]
        except:
            player_info["High School"] = "NA"
        all_players.append(player_info)
    return all_players

In [90]:
def all_years(sport):
    df = pd.DataFrame()
    for year in range(2011, 2024):
        if sport in ['mens-basketball', 'mens-golf', 'womens-tennis', 'womens-golf', 'womens-basketball']:
            season = str(year) + '-' + str(year + 1)[-2:]
        else:
            season = str(year)
        link = f"https://godeacs.com/sports/{sport}/roster/{season}"
        yearly = pd.DataFrame(get_roster(link))
        yearly['Year'] = year
        df = pd.concat([df, yearly])
    df['sport'] = sport
    return df

In [91]:
all_players = pd.DataFrame()
all_sports = ['mens-basketball', 'mens-golf', 'womens-tennis', 'womens-golf', 'womens-basketball', 'baseball', 'football', 'cross-country', 'mens-soccer', 'womens-soccer', 'field-hockey']
for sport in all_sports:
    all_players = pd.concat([all_players, all_years_both(sport)])

In [161]:
import copy
test = copy.deepcopy(all_players)
substrings_to_keep = [' Rs.', ' Sr.', ' Jr.', ' So.', ' Fr.', ' R-Fr.', ' R-So.', ' R-Jr.', ' Gr.', ' R-Sr.', ' 5th']
# all_players[all_players['Position'].str.contains('|'.join(substrings_to_keep))]
# test.loc[test['Position'].isin(substrings_to_keep), 'C']

In [164]:
test.loc[test['Position'].isin(substrings_to_keep), "Weight"] = test.loc[test['Position'].isin(substrings_to_keep), "Height"]

In [165]:
test.loc[test['Position'].isin(substrings_to_keep), "Height"] = test.loc[test['Position'].isin(substrings_to_keep), "Academic Year"]
test.loc[test['Position'].isin(substrings_to_keep), "Academic Year"] = test.loc[test['Position'].isin(substrings_to_keep), "Position"]

In [171]:
test.loc[test['Position'].isin(substrings_to_keep), "Position"] = "NA"

In [174]:
test.to_csv("All_Wake.csv")